#### 1. Загрузите данные ex9_movies.mat из файла.

In [21]:
import scipy.io as scio
import scipy.optimize as op
import numpy as np
import matplotlib.pyplot as plt
from surprise import BaselineOnly, Dataset, Reader, Trainset, KNNBaseline, SVD, accuracy
from surprise.model_selection import cross_validate, GridSearchCV

In [22]:
movies = scio.loadmat('C:/Users/Mikita/Desktop/1_Sem/MO/machine-learning-ex9/ex9_movies.mat')
y, r = movies['Y'], movies['R']

#### 2. Выберите число признаков фильмов (n) для реализации алгоритма коллаборативной фильтрации.

In [23]:
num_users = 4
num_movies = 5
num_features = 10

#### 3. Реализуйте функцию стоимости для алгоритма.
#### 5. При реализации используйте векторизацию для ускорения процесса обучения.
#### 6. Добавьте L2-регуляризацию в модель.

In [24]:
def cofiCostFunc(X, Theta, y, r, num_users, num_movies, num_features, l):
    J = 1 / 2 * np.sum(r * (X.dot(Theta.T) - y)**2) + l / \
        2 * (np.sum(Theta**2) + np.sum(X**2))
    return J

#### 4. Реализуйте функцию вычисления градиентов.

In [25]:
def cofiGradientFunc(X, Theta, y, r, num_users, num_movies, num_features, l):
    X_grad = (r * (X.dot(Theta.T) - y)).dot(Theta) + l * X
    Theta_grad = (r * (X.dot(Theta.T) - y)).T.dot(X) + l * Theta
    return X_grad, Theta_grad

#### 7. Добавьте несколько оценок фильмов от себя. Файл movie_ids.txt содержит индексы каждого из фильмов.

In [26]:
def loadMovieList():
    f = open('C:/Users/Mikita/Desktop/1_Sem/MO/machine-learning-ex9/movie_ids.txt').readlines()
    n = 1682
    list = [0]*1682
    i = 0
    for line in f:
        if (i+1)/10 < 1:
            list[i] = line[2:]
        elif (i+1)/100 < 1:
            list[i] = line[3:]
        elif (i+1)/1000 < 1:
            list[i] = line[4:]
        else:
            list[i] = line[5:]
        i += 1
    return list

In [27]:
movieList = loadMovieList()
my_ratings = np.zeros([1682,1])

my_ratings[540] = 4
my_ratings[383] = 5
my_ratings[27] = 4
my_ratings[66]= 5
my_ratings[49] = 4
my_ratings[88] = 5
my_ratings[126] = 3
my_ratings[131] = 3
my_ratings[1366] = 3
my_ratings[1158] = 5
my_ratings[719] = 4

for i in range(0,len(my_ratings)):
    if my_ratings[i] > 0:
        print('Rated %d for %s'%(my_ratings[i],movieList[i]),end="")
        

Rated 4 for Apollo 13 (1995)
Rated 4 for Star Wars (1977)
Rated 5 for Ace Ventura: Pet Detective (1994)
Rated 5 for Blade Runner (1982)
Rated 3 for Godfather, The (1972)
Rated 3 for Wizard of Oz, The (1939)
Rated 5 for Naked Gun 33 1/3: The Final Insult (1994)
Rated 4 for Mortal Kombat (1995)
Rated 4 for First Knight (1995)
Rated 5 for Stalker (1979)
Rated 3 for Faust (1994)


#### 8. Обучите модель с помощью градиентного спуска или других методов оптимизации.

In [28]:
def normalizeRatings(Y, R):
    m, n = np.shape(Y)
    Ymean = np.zeros([m, 1])
    Ynorm = np.zeros(np.shape(Y))
    for i in range(0,m):
        idx = np.where(R[i,:] == 1)
        Ymean[i] = np.mean(Y[i,idx])
        Ynorm[i,idx] = Y[i,idx] - Ymean[i]

    return Ynorm,Ymean

In [29]:
def cofiCostFunc(params, Y, R, num_users, num_movies,num_features, xlambda):
    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)

    # zero initial
    J = 0
    X_grad = np.zeros(np.shape(X))
    Theta_grad = np.zeros(np.shape(Theta))

    # start
    J = np.sum(R*((np.dot(X,Theta.T)-Y)**2))/2
    X_grad = np.dot((R*(np.dot(X,Theta.T)-Y)),Theta)
    Theta_grad = np.dot((R*(np.dot(X,Theta.T)-Y)).T,X)

    grad = np.r_[(X_grad.ravel().reshape(num_movies*num_features,1),Theta_grad.ravel().reshape(num_users*num_features,1))]
    return J,grad

In [30]:
ex8_movies = scio.loadmat('C:/Users/Mikita/Desktop/1_Sem/MO/machine-learning-ex9/ex9_movies.mat')

#  Y is a 1682x943 matrix, containing ratings (1-5) of 1682 movies by 943 users
#  R is a 1682x943 matrix, where R(i,j) = 1 if and only if user j gave a rating to movie i
R,Y = ex8_movies["R"], ex8_movies["Y"]

#  Add our own ratings to the data matrix
Y = np.c_[(my_ratings, Y)]
R = np.c_[(np.ceil(my_ratings / 5), R)]

#  Normalize Ratings
Ynorm, Ymean = normalizeRatings(Y, R)

#  Useful Values
num_users = Y.shape[1]
num_movies = Y.shape[0]
num_features = 10

# Set Initial Parameters (Theta, X)
X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

initial_parameters=np.r_[(X.ravel().reshape(num_movies * num_features, 1),Theta.ravel().reshape(num_users * num_features,1))]
xlambda = 10

theta = op.minimize(fun=cofiCostFunc, x0=initial_parameters,
                     args=(Ynorm, R, num_users, num_movies, num_features, xlambda),
                     method='TNC', jac=True, tol=1e-6, options={'maxiter':100, "disp":True}).x

# Unfold the returned theta back into U and W
X = theta[:num_movies * num_features].reshape(num_movies, num_features)
Theta = theta[num_movies * num_features:].reshape(num_users, num_features)

### 9. Сделайте рекомендации для себя. Совпали ли они с реальностью?

In [37]:
p = np.dot(X,Theta.T)
my_predictions = p[:,0].reshape(num_movies,1) + Ymean.reshape(num_movies,1)
my_predictions = my_predictions[:, 0]

movieList = loadMovieList()

my_pred_sort = np.sort(-my_predictions)
my_ix = np.argsort(-my_predictions)

print('\nTop recommendations for you:\n')
for i in range(0,20):
    j = int(my_ix[i])
    print('Predicting rating %.1f for movie %s'%(my_predictions[j], movieList[j]),end="")
    


Top recommendations for you:

Predicting rating 10.5 for movie Saint of Fort Washington, The (1993)
Predicting rating 10.2 for movie Nelly & Monsieur Arnaud (1995)
Predicting rating 9.7 for movie Walking and Talking (1996)
Predicting rating 9.4 for movie Innocents, The (1961)
Predicting rating 8.9 for movie Line King: Al Hirschfeld, The (1996)
Predicting rating 8.6 for movie Shaggy Dog, The (1959)
Predicting rating 8.3 for movie Man of the Year (1995)
Predicting rating 8.2 for movie Sirens (1994)
Predicting rating 8.2 for movie Star Kid (1997)
Predicting rating 8.0 for movie Mina Tannenbaum (1994)
Predicting rating 7.9 for movie Bitter Moon (1992)
Predicting rating 7.9 for movie Parent Trap, The (1961)
Predicting rating 7.9 for movie Fille seule, La (A Single Girl) (1995)
Predicting rating 7.8 for movie Belle de jour (1967)
Predicting rating 7.8 for movie Friday (1995)
Predicting rating 7.5 for movie Bewegte Mann, Der (1994)
Predicting rating 7.3 for movie Shooter, The (1995)
Predicti

In [44]:
#Отчасти эти фильмы похоже на те, что я положительно оценил

In [42]:
#10. Также обучите модель с помощью сингулярного разложения матриц. Отличаются ли полученные результаты?
 

In [33]:
# save *.csv file

Y_inlined = np.dstack([*np.indices(Y.T.shape), Y.T]).reshape(-1, 3)
np.savetxt('C:/Users/Mikita/Desktop/1_Sem/MO/machine-learning-ex9/ex9_movies.csv', Y_inlined[Y_inlined[:,2] != 0], fmt="%i %i %i")

# create dataset from *.csv file
reader = Reader(line_format='user item rating', sep=" ")
data = Dataset.load_from_file("C:/Users/Mikita/Desktop/1_Sem/MO/machine-learning-ex9/ex9_movies.csv", reader)

In [34]:
algo = SVD(n_factors=100, n_epochs=100, lr_all=0.1)
trainset = data.build_full_trainset()
testset = trainset.build_testset()

algo.fit(trainset)
p = algo.predict(uid=trainset.to_raw_uid(0), iid=trainset.to_raw_iid(0), r_ui=10)
predictions = algo.test(testset)

print(p)
print('Error = ', accuracy.rmse(predictions))

user: 0          item: 27         r_ui = 10.00   est = 3.99   {'was_impossible': False}
RMSE: 0.2470
Error =  0.2469551832692124


In [35]:
my_predictions = np.ndarray(1681)
for movie_index in np.arange(0, 1681):
    my_predictions[movie_index] = algo.predict(uid=trainset.to_raw_uid(0), iid=trainset.to_raw_iid(movie_index)).est

In [36]:
movieList = loadMovieList()

my_pred_sort = np.sort(-my_predictions)
my_ix = np.argsort(-my_predictions)

print('\nTop recommendations for you:\n')
for i in range(0,20):
    j = int(my_ix[i])
    print('Predicting rating %.1f for movie %s'%(my_predictions[j], movieList[j]),end="")
    


Top recommendations for you:

Predicting rating 5.0 for movie Raiders of the Lost Ark (1981)
Predicting rating 5.0 for movie Four Rooms (1995)
Predicting rating 5.0 for movie Get Shorty (1995)
Predicting rating 5.0 for movie Twelve Monkeys (1995)
Predicting rating 5.0 for movie Richard III (1995)
Predicting rating 4.9 for movie Clockwork Orange, A (1971)
Predicting rating 4.8 for movie Very Brady Sequel, A (1996)
Predicting rating 4.7 for movie Casablanca (1942)
Predicting rating 4.7 for movie Sting, The (1973)
Predicting rating 4.7 for movie Manhattan (1979)
Predicting rating 4.7 for movie Independence Day (ID4) (1996)
Predicting rating 4.7 for movie All About Eve (1950)
Predicting rating 4.6 for movie Cinema Paradiso (1988)
Predicting rating 4.6 for movie Desperado (1995)
Predicting rating 4.6 for movie Taxi Driver (1976)
Predicting rating 4.6 for movie Sgt. Bilko (1996)
Predicting rating 4.5 for movie Field of Dreams (1989)
Predicting rating 4.5 for movie Some Mother's Son (1996)
P

In [41]:
#Данные рекомендации больше подходят под те фильмы, что я смотрю